<a href="https://colab.research.google.com/github/anhle/tensorFlow2.x/blob/master/Chexpert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Attempting to replicate the results from:
CheXpert: A large chest radiograph dataset with uncertainty labels and expert comparison
https://arxiv.org/abs/1901.07031



# Download CheXpert Dataset

In [0]:
!pip install wget

  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [0]:
import zipfile
import wget
import os

dir_base = '/content' 
chexpert_url = 'http://download.cs.stanford.edu/deep/CheXpert-v1.0-small.zip' 
        
zipname = wget.download(chexpert_url,dir_base)

zip_file = zipfile.ZipFile(os.path.join(dir_base,zipname))
zip_file.extractall(dir_base)
zip_file.close()
    

In [0]:

import pandas as pd
import numpy as np
import keras
import tensorflow as tf
print(tf.__version__)

import datetime
from sklearn.metrics import roc_auc_score

%reload_ext autoreload
%autoreload 2
%matplotlib inline

1.13.1


## Load Data

## Uncertainty Approaches
The CheXpert paper outlines several different approaches to mapping using the uncertainty labels in the data:

* Ignoring - essentially removing from the calculation in the loss function
* Binary mapping - sending uncertain values to either 0 or 1
* Prevalence mapping - use the rate of prevelance of the feature as it's target value
* Self-training - consider the uncertain values as unlabeled
* 3-Class Classification - retain a separate value for uncertain and try to predict it as a class in its own right

The paper gives the results of different experiments with the above approaches and indicates the most accurate approach for each feature.

Approach/Feature	Atelectasis	Cardiomegaly	Consolidation	Edema	PleuralEffusion
U-Ignore	0.818(0.759,0.877)	0.828(0.769,0.888)	0.938(0.905,0.970)	0.934(0.893,0.975)	0.928(0.894,0.962)
U-Zeros	0.811(0.751,0.872)	0.840(0.783,0.897)	0.932(0.898,0.966)	0.929(0.888,0.970)	0.931(0.897,0.965)
U-Ones	0.858(0.806,0.910)	0.832(0.773,0.890)	0.899(0.854,0.944)	0.941(0.903,0.980)	0.934(0.901,0.967)
U-Mean	0.821(0.762,0.879)	0.832(0.771,0.892)	0.937(0.905,0.969)	0.939(0.902,0.975)	0.930(0.896,0.965)
U-SelfTrained	0.833(0.776,0.890)	0.831(0.770,0.891)	0.939(0.908,0.971)	0.935(0.896,0.974)	0.932(0.899,0.966)
U-MultiClass	0.821(0.763,0.879)	0.854(0.800,0.909)	0.937(0.905,0.969)	0.928(0.887,0.968)	0.936(0.904,0.967)

The binary mapping approaches (U-Ones and U-Zeros) are easiest to implement and so to begin with we take the best option between U-Ones and U-Zeros for each feature

* Atelectasis U-Ones
* Cardiomegaly U-Zeros
* Consolidation U-Zeros
* Edema U-Ones
* Pleural Effusion U-Zeros

In [0]:
# Path to images and category labels in data dir
_DATA_DIR = "/content/CheXpert-v1.0-small"
_IMAGE_DIR = './'
train_file = os.path.join(_DATA_DIR,'train.csv')
valid_file = os.path.join(_DATA_DIR,'valid.csv')

train_df = pd.read_csv(train_file)
valid_df = pd.read_csv(valid_file)

#Our classfication problem consists of only 5 classes as mentioned in the CheXpert Competition.
class_names = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']

train_df = train_df[train_df['Frontal/Lateral']=='Frontal']
train_path, y = train_df["Path"].as_matrix(), train_df[class_names]

train_counts = train_df.shape[0]
valid_counts = valid_df.shape[0]
print('train counts', train_counts)
print('valid counts', valid_counts)


train counts 191027
valid counts 234


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]


In [0]:
train_df.head(4)

In [0]:
y.head(4)

In [0]:
def get_sample_counts(file, class_names):
    """
    Get total and class-wise positive sample count of a dataset
    Arguments:
    output_dir - str, folder of dataset.csv
    dataset - str, train|dev|test
    class_names - list of str, target classes
    Returns:
    total_count - int
    class_positive_counts - dict of int, ex: {"Effusion": 300, "Infiltration": 500 ...}
    """
    df = pd.read_csv(file)
    total_count = df.shape[0]
    labels = df[class_names].as_matrix()
    positive_counts = np.sum(labels, axis=0)
    #class_positive_counts = dict(zip(class_names, positive_counts))
    class_positive_counts = {key:value for key, value in zip(class_names, positive_counts)}
    return total_count, class_positive_counts

def get_class_weights(total_counts, class_positive_counts, multiply):
    """
    Calculate class_weight used in training
    Arguments:
    total_counts - int
    class_positive_counts - dict of int, ex: {"Effusion": 300, "Infiltration": 500 ...}
    multiply - int, positve weighting multiply
    use_class_balancing - boolean
    Returns:
    class_weight - dict of dict, ex: {"Effusion": { 0: 0.01, 1: 0.99 }, ... }
    """
    def get_single_class_weight(pos_counts, total_counts):
        denominator = (total_counts - pos_counts) * multiply + pos_counts
        return {
            0: pos_counts / denominator,
            1: (denominator - pos_counts) / denominator,
        }

    class_names = list(class_positive_counts.keys())
    label_counts = np.array(list(class_positive_counts.values()))
    class_weights = []
    for i, class_name in enumerate(class_names):
        class_weights.append(get_single_class_weight(label_counts[i], total_counts))

    return class_weights

In [0]:
positive_weights_multiply=1
train_counts, train_pos_counts = get_sample_counts(train_file, class_names)
valid_counts, _ = get_sample_counts(valid_file, class_names)
class_weights = get_class_weights(
    train_counts,
    train_pos_counts,
    multiply=positive_weights_multiply,
)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [0]:
print('train count: ',train_counts)
print('valid count: ',valid_counts)

In [0]:
from PIL import Image
from skimage.transform import resize

class CheXpertDataGenerator(keras.utils.Sequence):
    'Data Generetor for CheXpert'

    def __init__(self, dataset_csv_file, class_names, source_image_dir, batch_size=16,
                 target_size=(224, 224), policy = "zeroes", augmenter=None, verbose=0,
                 shuffle_on_epoch_end=False, random_state=1):

        self.dataset_df = pd.read_csv(dataset_csv_file)
        self.source_image_dir = source_image_dir
        self.batch_size = batch_size
        self.target_size = target_size
        self.augmenter = augmenter
        self.verbose = verbose
        self.shuffle = shuffle_on_epoch_end
        self.random_state = random_state
        self.class_names = class_names
        self.policy = policy
        self.prepare_dataset()
        self.steps = int(np.ceil(len(self.x_path) / float(self.batch_size)))

        # print('steps..', self.steps)
    def __bool__(self):
        return True

    def __len__(self):
        return self.steps

    def __getitem__(self, idx):
        # print('idx....', idx)
        batch_x_path = self.x_path[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x = np.asarray([self.load_image(x_path) for x_path in batch_x_path])
        batch_x = self.transform_batch_images(batch_x)
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

    def load_image(self, image_file):
        image_path = os.path.join(self.source_image_dir, image_file)
        image = Image.open(image_path)
        image_array = np.asarray(image.convert("RGB"))
        image_array = image_array / 255.
        image_array = resize(image_array, self.target_size)
        return image_array

    def transform_batch_images(self, batch_x):
        if self.augmenter is not None:
            batch_x = self.augmenter.augment_images(batch_x)
        imagenet_mean = np.array([0.485, 0.456, 0.406])
        imagenet_std = np.array([0.229, 0.224, 0.225])
        if batch_x.shape == imagenet_mean.shape:
            batch_x = (batch_x - imagenet_mean) / imagenet_std
        return batch_x

    def get_y_true(self):
        if self.shuffle:
            raise ValueError("""
            You're trying run get_y_true() when generator option 'shuffle_on_epoch_end' is True.
            """)
        return self.y[:self.steps*self.batch_size, :]

    def prepare_dataset(self):
        self.dataset_df = self.dataset_df[self.dataset_df['Frontal/Lateral'] == 'Frontal']
        df = self.dataset_df.sample(frac=1., random_state=self.random_state)
        df.fillna(0, inplace=True)
        self.x_path, y_df = df["Path"].as_matrix(), df[self.class_names]
        self.class_ones = ['Atelectasis', 'Cardiomegaly']
        self.y = np.empty(y_df.shape, dtype=int)
        # print(y_ar.shape)
        for i, (index, row) in enumerate(y_df.iterrows()):
            labels = []
            for cls in self.class_names:
                #         print(cls)
                curr_val = row[cls]
                #         print(curr_val)
                feat_val = 0
                if curr_val:
                    curr_val = float(curr_val)
                    if curr_val == 1:
                        feat_val = 1
                    elif curr_val == -1:
                        if self.policy == "ones":
                            feat_val = 1
                        elif self.policy == "zeroes":
                            feat_val = 0
                        elif self.policy == "mixed":
                            if cls in self.class_ones:
                                feat_val = 1
                            else:
                                feat_val = 0
                        else:
                            feat_val = 0
                    else:
                        feat_val = 0
                else:
                    feat_val = 0
                labels.append(feat_val)
            # print(labels)
            self.y[i] = labels

    def on_epoch_end(self):
        if self.shuffle:
            self.random_state += 1
            self.prepare_dataset()

In [0]:
from imgaug import augmenters as iaa

augmenter = iaa.Sequential(
    [
        iaa.Fliplr(0.5),
        iaa.Affine(scale=(0.5, 1.5)),
        iaa.CropAndPad(percent=(-0.25, 0.25)),
        iaa.Noop(),
    ],
    random_order=True,
)

In [0]:
batch_size = 32
HEIGHT = 224
WIDTH = 224
_IMAGE_DIR = './'

train_data = CheXpertDataGenerator(dataset_csv_file=train_file,
            class_names=class_names,
            source_image_dir=_IMAGE_DIR,
            batch_size=batch_size,
            target_size=(HEIGHT, WIDTH),
            augmenter=augmenter,
            policy = 'mixed'
    )

valid_data = CheXpertDataGenerator(dataset_csv_file=valid_file,
            class_names=class_names,
            source_image_dir=_IMAGE_DIR,
            batch_size=batch_size,
            target_size=(HEIGHT, WIDTH),
            augmenter=None,
            policy ='mixed'
    )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:67: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [0]:

#from tensorflow.keras import backend as k
from keras.applications import DenseNet121
from keras import layers
from keras import models
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model

from tensorflow.python.keras import backend as k

import importlib

class ModelFactory:
    """
    Model facotry for Keras default models
    """

    def __init__(self):
        self.models_ = dict(
            DenseNet121=dict(
                input_shape=(224, 224, 3),
                module_name="densenet",
                last_conv_layer="bn",
            ),
            ResNet50=dict(
                input_shape=(224, 224, 3),
                module_name="resnet50",
                last_conv_layer="activation_49",
            ),
            InceptionV3=dict(
                input_shape=(299, 299, 3),
                module_name="inception_v3",
                last_conv_layer="mixed10",
            ),
            InceptionResNetV2=dict(
                input_shape=(299, 299, 3),
                module_name="inception_resnet_v2",
                last_conv_layer="conv_7b_ac",
            ),
            Xception=dict(
                input_shape=(224, 224, 3),
                pooling="avg"
            ),
            NASNetLarge=dict(
                input_shape=(331, 331, 3),
                module_name="nasnet",
                last_conv_layer="activation_260",
            ),
        )

    def get_last_conv_layer(self, model_name):
        return self.models_[model_name]["last_conv_layer"]

    def get_input_size(self, model_name):
        return self.models_[model_name]["input_shape"][:2]

    def get_model(self, class_names, model_name="DenseNet121", use_base_weights=True,
                  weights_path=None, input_shape=None):

        if use_base_weights is True:
            base_weights = "imagenet"
        else:
            base_weights = None

        base_model_class = getattr(
            importlib.import_module(
                "keras.applications." + self.models_[model_name]['module_name']
            ),
            model_name)

        if input_shape is None:
            input_shape = self.models_[model_name]["input_shape"]

        img_input = Input(shape=input_shape)

        base_model = base_model_class(
            include_top=False,
            input_tensor=img_input,
            input_shape=input_shape,
            weights=base_weights,
            pooling="avg")
        x = base_model.output
        predictions = Dense(len(class_names), activation="sigmoid", name="predictions")(x)
        model = Model(inputs=img_input, outputs=predictions)

        if weights_path == "":
            weights_path = None

        if weights_path is not None:
            print("load model weights_path", weights_path)
            model.load_weights(weights_path)
        return model


def DenseNet(height, width, channels, classes):
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(height, width, channels))
    for layer in base_model.layers[-4:]:
        layer.trainable = False
    x = base_model.layers[-1].output  # es la salida del ultimo activation despues del add
    x = layers.GlobalAveragePooling2D()(x)
    #x = layers.GlobalMaxPool2D()(x)
    # output layer
    #---1) NO LINEAL + LINEAL
    # prepredictions = Dense(256, activation='relu')(x)
    # predictions = Dense(classes, activation='sigmoid')(prepredictions)

    #---2) LINEAL
    predictions = Dense(classes, activation='sigmoid')(x)

    model = models.Model(inputs=base_model.input, outputs=predictions)
    model.summary()
    return model

In [0]:
model_file_path = '/content/best_weights_1555982768.7076797.h5'
model_type = "DenseNet121"
learning_rate = 1e-3

model_factory = ModelFactory()

model = model_factory.get_model(
                        class_names,
                        model_name=model_type,
                        use_base_weights=True,
                        weights_path=model_file_path,
                        input_shape=(HEIGHT, WIDTH, 3)
)

optimizer = keras.optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy", "binary_accuracy"])

load model weights_path /content/best_weights_1555982768.7076797.h5


In [0]:
from keras.callbacks import Callback
from sklearn.metrics import roc_auc_score
import keras.backend as kb
import json

import shutil

class MultipleClassAUROC(Callback):
    """
    Monitor mean AUROC and update model
    """
    def __init__(self, sequence, class_names, weights_path, stats=None, workers=1):
        super(Callback, self).__init__()
        self.sequence = sequence
        self.workers = workers
        self.class_names = class_names
        self.weights_path = weights_path
        self.best_weights_path = os.path.join(
            os.path.split(weights_path)[0],"best_"+os.path.split(weights_path)[1],
        )
        self.best_auroc_log_path = os.path.join(
            os.path.split(weights_path)[0],
            "best_auroc.log",
        )
        self.stats_output_path = os.path.join(
            os.path.split(weights_path)[0],
            ".training_stats.json"
        )
        # for resuming previous training
        if stats:
            self.stats = stats
        else:
            self.stats = {"best_mean_auroc": 0}

        # aurocs log
        self.aurocs = {}
        for c in self.class_names:
            self.aurocs[c] = []

    def on_epoch_end(self, epoch, logs={}):
        """
        Calculate the average AUROC and save the best model weights according
        to this metric.
        """
        print("\n*********************************")
        self.stats["lr"] = float(kb.eval(self.model.optimizer.lr))
        print("current learning rate:", self.stats['lr'])

        """
        y_hat shape: (#samples, len(class_names))
        y: [(#samples, 1), (#samples, 1) ... (#samples, 1)]
        """
        y_hat = self.model.predict_generator(self.sequence, workers=self.workers)
        y = self.sequence.get_y_true()

        # print("****************************")
        # print('y_hat', y_hat.shape)
        # print(y_hat)
        # print('y', y.shape)
        # print(y)
        # print("****************************")

        print("*** Epoch# %d dev auroc ***" % (epoch + 1))
        current_auroc = []
        for i in range(len(self.class_names)):
            try:
                score = roc_auc_score(y[:, i], y_hat[:, i])
            except ValueError:
                score = 0
            self.aurocs[self.class_names[i]].append(score)
            current_auroc.append(score)
            print("%d. %s: %f" % ((i+1), self.class_names[i], score))
        print("*********************************")

        # customize your multiple class metrics here
        mean_auroc = np.mean(current_auroc)
        print("mean auroc: %f" % (mean_auroc))
        if mean_auroc > self.stats["best_mean_auroc"]:
            print("Update best auroc from %f to %f" % (self.stats['best_mean_auroc'], mean_auroc))

            # 1. copy best model
            shutil.copy(self.weights_path, self.best_weights_path)

            # # # 2. update log file
            # print("update log file:", self.best_auroc_log_path)
            # with open(self.best_auroc_log_path, "a") as f:
            #     f.write("epoch #%d auroc: %f, lr: %f \n" % ((epoch+1), mean_auroc, self.stats['lr']))

            # 3. write stats output, this is used for resuming the training
            with open(self.stats_output_path, 'w') as f:
                json.dump(self.stats, f)

            print("Update model file: %s -> %s" % (self.weights_path, self.best_weights_path))
            self.stats["best_mean_auroc"] = mean_auroc
            print("*********************************")
        return


In [0]:
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
output_weights_path = '/content/weights.h5'
training_stats = {}
generator_workers = 8
minimum_lr = 1e-8
patience_reduce_lr = 2

auroc = MultipleClassAUROC(
        sequence=valid_data,
        class_names=class_names,
        weights_path=output_weights_path,
        stats=training_stats,
        workers=generator_workers,
    )
checkpoint = ModelCheckpoint(
    output_weights_path,
    save_weights_only=True,
    save_best_only=True,
    verbose=1,
)
callbacks = [
    checkpoint,
    TensorBoard(log_dir="/content", batch_size=batch_size),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=patience_reduce_lr,
                      verbose=1, mode="min", min_lr=minimum_lr),
    auroc,
]

In [0]:
epochs = 5
class_weights = 1
history = model.fit_generator(
        generator=train_data,
        steps_per_epoch=None,
        epochs=epochs,
        validation_data=valid_data,
        validation_steps=None,
        callbacks=callbacks,
        workers=generator_workers,
        shuffle=False,
    )

with open('out.pkl', 'wb') as f:
        pickle.dump({
            'history': history.history,
            'auroc': auroc.aurocs,
        }, f)

Epoch 1/5


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


5970/5970 [==============================] - 3425s 574ms/step - loss: 0.4129 - acc: 0.8071 - binary_accuracy: 0.8071 - val_loss: 0.5362 - val_acc: 0.7822 - val_binary_accuracy: 0.7822

Epoch 00001: val_loss improved from 0.57778 to 0.53618, saving model to /content/weights.h5

*********************************
current learning rate: 0.0010000000474974513
*** Epoch# 1 dev auroc ***
1. Atelectasis: 0.755486
2. Cardiomegaly: 0.811052
3. Consolidation: 0.899816
4. Edema: 0.908482
5. Pleural Effusion: 0.915534
*********************************
mean auroc: 0.858074
Update best auroc from 0.000000 to 0.858074


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Update model file: /content/weights.h5 -> /content/best_weights.h5
*********************************
Epoch 2/5
5970/5970 [==============================] - 3426s 574ms/step - loss: 0.4112 - acc: 0.8079 - binary_accuracy: 0.8079 - val_loss: 0.4538 - val_acc: 0.7921 - val_binary_accuracy: 0.7921

Epoch 00002: val_loss improved from 0.53618 to 0.45381, saving model to /content/weights.h5

*********************************
current learning rate: 0.0010000000474974513
*** Epoch# 2 dev auroc ***
1. Atelectasis: 0.821837
2. Cardiomegaly: 0.800134
3. Consolidation: 0.920956
4. Edema: 0.918304
5. Pleural Effusion: 0.921875
*********************************
mean auroc: 0.876621
Update best auroc from 0.858074 to 0.876621
Update model file: /content/weights.h5 -> /content/best_weights.h5
*********************************
Epoch 3/5


...

5970/5970 [==============================] - 3432s 575ms/step - loss: 0.4095 - acc: 0.8084 - binary_accuracy: 0.8084 - val_loss: 0.4611 - val_acc: 0.7842 - val_binary_accuracy: 0.7842

Epoch 00003: val_loss did not improve from 0.45381

*********************************
current learning rate: 0.0010000000474974513
*** Epoch# 3 dev auroc ***
1. Atelectasis: 0.744777
2. Cardiomegaly: 0.825312
3. Consolidation: 0.825184
4. Edema: 0.886161
5. Pleural Effusion: 0.906929
*********************************
mean auroc: 0.837673
Epoch 4/5


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


5970/5970 [==============================] - 3445s 577ms/step - loss: 0.4084 - acc: 0.8094 - binary_accuracy: 0.8094 - val_loss: 0.4777 - val_acc: 0.7911 - val_binary_accuracy: 0.7911

Epoch 00004: val_loss did not improve from 0.45381

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

*********************************
current learning rate: 0.00010000000474974513
*** Epoch# 4 dev auroc ***
1. Atelectasis: 0.750236
2. Cardiomegaly: 0.816065
3. Consolidation: 0.932169
4. Edema: 0.918452
5. Pleural Effusion: 0.908062
*********************************
mean auroc: 0.864997
Epoch 5/5


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


5660/5970 [===========================>..] - ETA: 2:59 - loss: 0.4000 - acc: 0.8134 - binary_accuracy: 0.8134